In [1]:
import numpy as np
import copy
import torch
from transformers import logging
logging.set_verbosity_error()

from PIL import Image
import matplotlib.pyplot as plt

from fromage import models
from fromage import utils

/home/ubuntu/anaconda3/envs/lavis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load model used in the paper.
model_dir = '/home/ubuntu/Project/Project_final/11-777-Project/fromage/runs/frozen_10'
model = models.load_fromage(model_dir,type1='ss')

Using HuggingFace AutoFeatureExtractor for openai/clip-vit-large-patch14.
Using facebook/opt-1.3b for the language model.
Using openai/clip-vit-large-patch14 for the visual model with 4 visual tokens.


/home/ubuntu/anaconda3/envs/lavis/lib/python3.8/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


Freezing the LM.


/home/ubuntu/anaconda3/envs/lavis/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [3]:
import json
import random
import os
dataset1 = json.load(open("/home/ubuntu/Project/11-777-Project-aa/Data/text_image_queries_final_1.json", "r"))
dataset2 = json.load(open("/home/ubuntu/Project/11-777-Project-aa/Data/text_image_queries_final_2.json", "r"))
dataset3 = json.load(open("/home/ubuntu/Project/11-777-Project-aa/Data/text_image_queries_final_with_subanswers.json", "r"))

In [4]:
len(dataset1), len(dataset2), len(dataset3)

(38, 38, 38)

In [5]:
def generate_answer(q1,q2,c1,c2,Q):
    prompt1= c1+c2+ 'Q: '+  q1+ '\nA:'
    A1=model.generate_for_images_and_texts([prompt1], num_words=40)
    prompt2= c1+c2+ 'Q: '+  q2+ '\nA:'
    A2=model.generate_for_images_and_texts([prompt2], num_words=40)
    prompt3= 'Q: '+  q1+ ' \nA: ' +A1[0] + '\nQ: '+  q2+ ' \nA: ' +A2[0] + '\nQ: '+  Q+ ' \nA:'
    A_final=model.generate_for_images_and_texts([prompt3], num_words=40)
    return A_final[0]

In [6]:
import pickle
import os
def generate_answer_img(q1,q2,c1,c2,Q):
    if c2==None:
        gold_feature = []
        imgid_map=pickle.load(open('/home/ubuntu/Project/11-777-Project-aa/Data/image_id_map_0328.pkl', "rb"))
        image_id1 = imgid_map[c1]
        image_id2 = imgid_map[c1]
        if os.path.exists(os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('dev', (image_id1%10000000)//1000, image_id1))):
            image_feature_path1='/home/ubuntu/Project/11-777-Project-aa/Data/dev/{}/{}.pkl'.format((image_id1%10000000)//1000, image_id1)
        if os.path.exists(os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('train', (image_id1%10000000)//1000, image_id1))):
            image_feature_path1='/home/ubuntu/Project/11-777-Project-aa/Data/dev/{}/{}.pkl'.format((image_id1%10000000)//1000, image_id1)
        if os.path.exists(os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('test', (image_id1%10000000)//1000, image_id1))):
            image_feature_path1='/home/ubuntu/Project/11-777-Project-aa/Data/dev/{}/{}.pkl'.format((image_id1%10000000)//1000, image_id1)
        if os.path.exists(os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('dev', (image_id2%10000000)//1000, image_id2))):
            image_feature_path2='/home/ubuntu/Project/11-777-Project-aa/Data/dev/{}/{}.pkl'.format((image_id2%10000000)//1000, image_id2)
        if os.path.exists(os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('train', (image_id2%10000000)//1000, image_id2))):
            image_feature_path2='/home/ubuntu/Project/11-777-Project-aa/Data/dev/{}/{}.pkl'.format((image_id2%10000000)//1000, image_id2)
        if os.path.exists(os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('test', (image_id2%10000000)//1000, image_id2))):
            image_feature_path2='/home/ubuntu/Project/11-777-Project-aa/Data/dev/{}/{}.pkl'.format((image_id2%10000000)//1000, image_id2)
        gold_feature.append(image_feature_path1)
        gold_feature.append(image_feature_path2)
        c=[gold_feature]
        prompt1= 'Q: '+  q1+ '\nA:'
        A1=model.generate_for_images_and_texts([c,prompt1], num_words=40)
        # prompt2= 'Q: '+  q2+ '\nA:'
        # A2=model.generate_for_images_and_texts([c,prompt2], num_words=40)
        # prompt3= 'Q: '+  q1+ ' \nA: ' +A1[0] + '\nQ: '+  q2+ ' \nA: ' +A2[0] + '\nQ: '+  Q+ ' \nA:'
        # A_final=model.generate_for_images_and_texts([prompt3], num_words=40)
        A_final=A1
    else:
        gold_feature = []
        imgid_map=pickle.load(open('/home/ubuntu/Project/11-777-Project-aa/Data/image_id_map_0328.pkl', "rb"))
        image_id1 = imgid_map[c1]
        image_id2 = imgid_map[c2]
        if os.path.exists(os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('dev', (image_id1%10000000)//1000, image_id1))):
            image_feature_path1='/home/ubuntu/Project/11-777-Project-aa/Data/dev/{}/{}.pkl'.format((image_id1%10000000)//1000, image_id1)
        if os.path.exists(os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('train', (image_id1%10000000)//1000, image_id1))):
            image_feature_path1='/home/ubuntu/Project/11-777-Project-aa/Data/dev/{}/{}.pkl'.format((image_id1%10000000)//1000, image_id1)
        if os.path.exists(os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('test', (image_id1%10000000)//1000, image_id1))):
            image_feature_path1='/home/ubuntu/Project/11-777-Project-aa/Data/dev/{}/{}.pkl'.format((image_id1%10000000)//1000, image_id1)
        if os.path.exists(os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('dev', (image_id2%10000000)//1000, image_id2))):
            image_feature_path2='/home/ubuntu/Project/11-777-Project-aa/Data/dev/{}/{}.pkl'.format((image_id2%10000000)//1000, image_id2)
        if os.path.exists(os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('train', (image_id2%10000000)//1000, image_id2))):
            image_feature_path2='/home/ubuntu/Project/11-777-Project-aa/Data/dev/{}/{}.pkl'.format((image_id2%10000000)//1000, image_id2)
        if os.path.exists(os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('test', (image_id2%10000000)//1000, image_id2))):
            image_feature_path2='/home/ubuntu/Project/11-777-Project-aa/Data/dev/{}/{}.pkl'.format((image_id2%10000000)//1000, image_id2)
        gold_feature.append(image_feature_path1)
        gold_feature.append(image_feature_path2)
        c=[gold_feature]
        prompt1= 'Q: '+  q1+ '\nA:'
        A1=model.generate_for_images_and_texts([c,prompt1], num_words=40)
        prompt2= 'Q: '+  q2+ '\nA:'
        A2=model.generate_for_images_and_texts([c,prompt2], num_words=40)
        prompt3= 'Q: '+  q1+ ' \nA: ' +A1[0] + '\nQ: '+  q2+ ' \nA: ' +A2[0] + '\nQ: '+  Q+ ' \nA:'
        A_final=model.generate_for_images_and_texts([prompt3], num_words=40)
    return A_final[0]

In [7]:
Answers_real_im1 = []
Answers_pred_im1 = []
for i in dataset1:
    datum = dataset1[i]
    if len(datum['txt_posFacts']) == 0:
        q1= dataset1[i]['Q']
        q2= dataset2[i]['Q']
        if(len(dataset1[i]['img_posFacts'])==1):
            c1= dataset1[i]['img_posFacts'][0]['image_id']
            c2= dataset1[i]['img_posFacts'][0]['image_id']
        else:
            continue
        Q= dataset3[i]['Q']
        A= dataset3[i]['A'][0]
        Answers_real_im1.append(A)
        Answers_pred_im1.append(generate_answer_img(q1,q2,c1,c2,Q))

In [8]:
len(Answers_real_im1), len(Answers_pred_im1)

(11, 11)

In [9]:
Answers_real_im = []
Answers_pred_im = []
for i in dataset1:
    datum = dataset1[i]
    if len(datum['txt_posFacts']) == 0:
        q1= dataset1[i]['Q']
        q2= dataset2[i]['Q']
        if(len(dataset1[i]['img_posFacts'])<2):
            continue
        c1= dataset1[i]['img_posFacts'][0]['image_id']
        c2= dataset1[i]['img_posFacts'][1]['image_id']
        Q= dataset3[i]['Q']
        A= dataset3[i]['A'][0]
        Answers_real_im.append(A)
        Answers_pred_im.append(generate_answer_img(q1,q2,c1,c2,Q))

In [10]:
len(Answers_real_im), len(Answers_pred_im)

(14, 14)

In [11]:
Answers_real = []
Answers_pred = []
for i in dataset1:
    datum = dataset1[i]
    if len(datum['txt_posFacts']) != 0:
        q1= dataset1[i]['Q']
        q2= dataset2[i]['Q']
        c1= dataset1[i]['txt_posFacts'][0]['fact']
        c2= dataset2[i]['txt_posFacts'][1]['fact']
        Q= dataset3[i]['Q']
        A= dataset3[i]['A'][0]
        Answers_real.append(A)
        Answers_pred.append(generate_answer(q1,q2,c1,c2,Q))


In [12]:
len(Answers_real), len(Answers_pred)

(13, 13)

In [6]:
import pickle
import os
gold_feature = []
imgid_map=pickle.load(open('/home/ubuntu/Project/11-777-Project-aa/Data/image_id_map_0328.pkl', "rb"))
# image_id = imgid_map[image_id]
# image_feature_path =os.path.join("/home/ubuntu/Project/11-777-Project-aa/Data/", "{}/{}/{}.pkl".format('dev', (image_id%10000000)//1000, image_id))
image_feature_path='/home/ubuntu/Project/11-777-Project-aa/Data/dev/41/40041882.pkl'
gold_feature.append(image_feature_path)
gold_feature.append(image_feature_path)


In [13]:
A_r=Answers_real+Answers_real_im+Answers_real_im1

In [14]:
A_p=Answers_pred+Answers_pred_im+Answers_pred_im1

In [17]:
#save the answers in pkl
with open('Answers_real.pkl', 'wb') as f:
    pickle.dump(A_r, f)
with open('Answers_pred_obj.pkl', 'wb') as f:
    pickle.dump(A_p, f)

In [2]:
# load the pickle files
import pickle
with open('Answers_pred_obj.pkl', 'rb') as f:
    Answers = pickle.load(f)


In [4]:
Answers[14:]

['  "Yes, there are steps at the entrance to the Hood Museum of Art and Budynek Biblioteki.',
 '  "No, it\'s not.',
 '  "Blue"I\'m not sure if this is a joke or not.',
 '  "No, the seats at Baku Olympic Stadium are not the same color as the seats in the stadium at Olympic Park in Munich.',
 '  "No, they are both located on the ground floor of the building.',
 '  "No, it is not.',
 '  The pedestals are the places where the statues are placed.',
 '  "Yes, but the spiky leaves are more common.',
 '  "A bear.',
 ' The Architecture Building is a building that is designed to be a home for the people who live in it.',
 '  "No, they shoot with their left hands.',
 '  Yes.',
 '  "They\'re not headlights, they\'re headlights.',
 '  "Yes, the traditional dress of Sardinian women includes a skirt or pants.',
 ' Q: "How many total light posts are at both ends in London?"',
 '  The petals of the flower of the Camellia Japonica are arranged in a circle, with the petals of the flower of the Camellia J

In [23]:

for inp_text in ['The human history of Western Australia commenced between 40,000 and 60,000 years ago with the arrival of Indigenous Australians on the northwest coast. The first inhabitants expanded across the east and south of the continent. The North Shore is a region of Sydney, New South Wales, Australia, a subset of the Northern Sydney region. The region is customary, not legal or administrative, and generally refers to the suburbs located on the northern side of Sydney Harbour up to Waitara, which are between Middle Harbour and the Lane Cove River. Q: what is the name of the continent contains sydney harbour?\nA:']:
    prompt = [inp_text]
    model_outputs = model.generate_for_images_and_texts(prompt, num_words=40)

In [24]:
model_outputs 

[' Australia']

In [ ]:
a_r_t=['Australia.',
 'Think and Walking Away were released two years apart.',
 'West Virginia',
 'They are the same gender.',
 'teeth',
 'Genetics',
 'Pulmonary fibrosis affects the lungs, while Myelofibrosis affects the bone marrow.',
 'They are both an example of subfamily, under the larger name of Felidae.',
 'The West Rail line and the Mass Transit Railway both use light rails.',
 'The Palatine Hill',
 'No.',
 'No',
 'Both nerves are part of limbs.']

In [13]:
answer_pred_obj=Answers_pred_im+Answers_pred_im1+Answers_pred


In [14]:
#save the answers as pkl
import pickle
with open('answer_pred_obj.pkl', 'wb') as f:
    pickle.dump(answer_pred_obj, f)

In [15]:
answer_real=Answers_real+Answers_real_im+Answers_real_im1

In [16]:
import pickle
with open('answer_real.pkl', 'wb') as f:
    pickle.dump(answer_real, f)

In [ ]:
5